# Bogolyubov approximation for diagonal model of an interacting Bose gas

$$
\rho_{0} = \frac{1}{V} \frac{\sum_{n=1}^\infty = ne^{\beta\{(- \lambda + \mu + \frac{\gamma}{V})n - \frac{\lambda}{V}n^{2}\}}}{\sum_{n=0}^\infty = e^{\beta\{(- \lambda + \mu + \frac{\gamma}{V})n - \frac{\lambda}{V}n^{2}\}}} \sim (\frac{\mu - \lambda}{2 \gamma})
$$

### Observación parametros

__γ__,__β__ son parámetros positivos. 

$λ < µ < 0 $ y __V__ muy grande.

## *Desarrollo*

### Dependencias

In [49]:
import math
import matplotlib.pyplot as plt
import time
import sys

$$
\sum_{n=1}^\infty = ne^{\beta\{(- \lambda + \mu + \frac{\gamma}{V})n - \frac{\lambda}{V}n^{2}\}}
$$

In [28]:
def f1(b , u , lam , gam , v , n):
    sumatoria = 0
    for i in range(1, n):
        sumatoria+=n*math.exp(b*((-lam+u+(gam/v))*n-(lam/v)*math.pow(n, 2)))
    return sumatoria

$$
V \sum_{n=0}^\infty = e^{\beta\{(- \lambda + \mu + \frac{\gamma}{V})n - \frac{\lambda}{V}n^{2}\}}
$$

In [29]:
def f2(b , u , lam , gam , v , n):
    sumatoria = 0
    
    for i in range(n):
        sumatoria+=math.exp(b*((-lam+u+(gam/v))*n-(lam/v)*math.pow(n, 2)))
        
    resultado = v * sumatoria
    return resultado

$$
\frac{\mu - \lambda}{2 \gamma} 
$$

In [19]:
def validador(u , lam , gam):
    resultado = (u-lam)/(2*gam)
    return resultado

### Parámetros formula

In [36]:
B = 300  
U = 0.5
lam = 0.05
gam = 5
V = 500
N = 55

### Obtención de *ρ* y *validador* (r)

In [60]:
try:
    ro =  f1(B, U, lam, gam, V, N)/f2(B, u, lam, gam, V, N)

except ZeroDivisionError:
    print('Division por cero')

r = validador(U,lam,gam)

print ("Valor ro: ",ro)
print ("Valor r: ",r)

#Da error de overflow

OverflowError: math range error

In [39]:
def argumento(b , u , lam , gam , v , n):
    return b*((-lam+u+(gam/v))*n-(lam/v)*math.pow(n, 2))

In [42]:
result=argumento(B,U,lam,gam,V,N)
print (result)

7499.250000000001


In [43]:
math.exp(result)

OverflowError: math range error

In [50]:
math.log(sys.float_info.max)

709.782712893384

In [51]:
sys.float_info.max

1.7976931348623157e+308

In [53]:
math.exp(math.log(sys.float_info.max))

1.7976931348622732e+308

http://stackoverflow.com/questions/4050907/python-overflowerror-math-range-error

https://docs.python.org/3.6/library/math.html